# <span style="color: lightgreen;">DS310 Homework 8 - Microsoft Power BI - Data Generator</span>

---

### <span style="color: red;">Execute This Jupyter Notebook to Generate Unique Data for PowerBI Homework Submission.</span>

This procedure will produce random data under defined rules, ensuring that each student's submission remains unique.

Additionally, refrain from attempting to analyze the data, as it is entirely random and lacks meaningful interpretation.

**Utilize the data solely for visualization purposes.**

In [1]:
# INSTALL NECESSARY PACKAGES TO YOUR ENVIRONMENT

!pip install numpy pandas
!pip install XlsxWriter

In [2]:
import pandas as pd
import numpy as np

# Constants
num_universities = 20
num_students = np.random.randint(20000, 50000)
num_courses_per_university = np.random.randint(50, 100, num_universities)
num_professors_per_university = np.random.randint(50, 100, num_universities)
min_enrollments_per_student = 1
max_enrollments_per_student = 10
countries = ["France", "Spain", "United States", "South Korea", "Italy", "Turkey", "Mexico", "Germany", "Thailand", "United Kingdom"]

# COUNTRY Table
country_df = pd.DataFrame({
    'Country_ID': np.arange(1, len(countries) + 1),
    'Country_Name': countries
})

# UNIVERSITIES Table
universities = pd.DataFrame({
    'University_ID': np.arange(1, num_universities + 1),
    'University_Name': [f'University {i}' for i in range(1, num_universities + 1)],
    'Country_ID': np.random.choice(country_df['Country_ID'], num_universities),
    'Establishment_Year': np.random.randint(1800, 2000, num_universities),
    'Type': ['Public' if year < 1900 else np.random.choice(['Public', 'Private']) for year in np.random.randint(1800, 2000, num_universities)]
})

# PROFESSORS Table
total_professors = num_professors_per_university.sum()
professors = pd.DataFrame({
    'Professor_ID': np.arange(1, total_professors + 1),
    'Professor_Name': [f'Professor {i}' for i in range(1, total_professors + 1)],
    'Years_of_Experience': np.random.randint(1, 30, total_professors),
    'Rank': ['Full' if exp > 20 else 'Associate' if exp > 10 else 'Assistant' for exp in np.random.randint(1, 30, total_professors)],
    'Gender': np.random.choice(["Male", "Female"], total_professors)
})

# MAJORS Table
majors = pd.DataFrame({
    'Major_ID': np.arange(1, 16),
    'Major_Name': ["Business Administration", "Computer Science", "Nursing", "Engineering", "Biology", 
                   "Education", "Psychology", "Communications", "Accounting", "Political Science", 
                   "Economics", "English", "Finance", "Marketing", "Data Science"],
    'Duration_Years': np.random.choice([3, 4, 5], 15)
})

# COURSES Table
total_courses = num_courses_per_university.sum()
courses = pd.DataFrame({
    'Course_ID': np.arange(1, total_courses + 1),
    'University_ID': np.repeat(np.arange(1, num_universities + 1), num_courses_per_university[:num_universities]),
    'Professor_ID': np.random.choice(professors['Professor_ID'], total_courses),
    'Course_Name': [f'Course {i}' for i in range(1, total_courses + 1)],
    'Credits': np.random.randint(1, 5, total_courses),
    'Level': np.random.choice(['Undergraduate', 'Graduate'], total_courses)
})

# STUDENTS Table
students = pd.DataFrame({
    'Student_ID': np.arange(1, num_students + 1),
    'Student_Name': [f'Student {i}' for i in range(1, num_students + 1)],
    'Major_ID': np.random.choice(majors['Major_ID'], num_students),
    'Age': np.random.randint(18, 35, num_students),
    'Gender': np.random.choice(["Male", "Female"], num_students),
    'Nationality': np.random.choice(country_df['Country_Name'], num_students)
})

# DATE Table
date_table = pd.DataFrame({
    'Date': pd.date_range(start='2019-01-01', end='2023-12-31')
})
date_table = date_table.assign(
    Day=lambda x: x.Date.dt.day,
    Month=lambda x: x.Date.dt.month_name(),
    Year=lambda x: x.Date.dt.year,
    Quarter=lambda x: x.Date.dt.quarter,
    Weekday=lambda x: x.Date.dt.day_name(),
    Is_Weekend=lambda x: x.Date.dt.weekday > 4
)
date_table['Fiscal_Quarter'] = 'FQ' + date_table['Quarter'].astype(str)
date_table['Fiscal_Year'] = date_table['Year']
date_table['Full_Date'] = date_table['Date'].dt.strftime('%Y-%m-%d')
date_table['MonthKey'] = date_table['Date'].dt.strftime('%Y%m')
date_table['DateKey'] = date_table['Date'].dt.strftime('%Y%m%d')

# ENROLLMENT Table
enrollment_list = []
for student_id in students['Student_ID']:
    num_enrollments = np.random.randint(min_enrollments_per_student, max_enrollments_per_student + 1)
    chosen_courses = np.random.choice(courses['Course_ID'], num_enrollments, replace=False)
    for course_id in chosen_courses:
        chosen_date = np.random.choice(date_table['Date'])
        date_key = pd.Timestamp(chosen_date).strftime('%Y%m%d')
        enrollment_list.append({
            'Enrollment_ID': len(enrollment_list) + 1,
            'Student_ID': student_id,
            'Course_ID': course_id,
            'DateKey': date_key,
            'Enrollment_Cost': np.random.uniform(100, 1000)
        })

enrollments = pd.DataFrame(enrollment_list)

# SAVE AS EXCEL FILE
excel_file_path = 'University_Course_Enrollment_Data.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    country_df.to_excel(writer, sheet_name='Countries', index=False)
    universities.to_excel(writer, sheet_name='Universities', index=False)
    courses.to_excel(writer, sheet_name='Courses', index=False)
    students.to_excel(writer, sheet_name='Students', index=False)
    enrollments.to_excel(writer, sheet_name='Enrollments', index=False)
    professors.to_excel(writer, sheet_name='Professors', index=False)
    majors.to_excel(writer, sheet_name='Majors', index=False)
    date_table.to_excel(writer, sheet_name='Date', index=False)

# PRINT IF SUCCESSFUL
completed_message = f"Excel file created at {excel_file_path}"
print(completed_message)


Excel file created at University_Course_Enrollment_Data.xlsx


## Data Dictionary

### <span style="color: red;">Execute the Cell to Access the Dictionary</span>

In [3]:
# INSTALL NECESSARY PACKAGES TO YOUR ENVIRONMENT

!pip install fpdf

In [4]:
from fpdf import FPDF
import pandas as pd

# Dataframes created in the previous code
country_df = pd.DataFrame({
    'Country_ID': ["Unique identifier for each country"],
    'Country_Name': ["Name of the country"]
})

universities = pd.DataFrame({
    'University_ID': ["Unique identifier for each university"],
    'University_Name': ["Name of the university"],
    'Country_ID': ["Identifier for the country where the university is located"],
    'Establishment_Year': ["Year when the university was established"],
    'Type': ["Type of the university (Public/Private)"]
})

professors = pd.DataFrame({
    'Professor_ID': ["Unique identifier for each professor"],
    'Professor_Name': ["Name of the professor"],
    'Years_of_Experience': ["Number of years of teaching experience"],
    'Rank': ["Academic rank of the professor (Full/Associate/Assistant)"],
    'Gender': ["Gender of the professor"]
})

majors = pd.DataFrame({
    'Major_ID': ["Unique identifier for each major"],
    'Major_Name': ["Name of the major"],
    'Duration_Years': ["Duration of the major program in years"]
})

courses = pd.DataFrame({
    'Course_ID': ["Unique identifier for each course"],
    'University_ID': ["Identifier for the university offering the course"],
    'Professor_ID': ["Identifier for the professor teaching the course"],
    'Course_Name': ["Name of the course"],
    'Credits': ["Number of credits for the course"],
    'Level': ["Academic level of the course (Undergraduate/Graduate)"]
})

students = pd.DataFrame({
    'Student_ID': ["Unique identifier for each student"],
    'Student_Name': ["Name of the student"],
    'Major_ID': ["Identifier for the major chosen by the student"],
    'Age': ["Age of the student"],
    'Gender': ["Gender of the student"],
    'Nationality': ["Nationality of the student"]
})

enrollments = pd.DataFrame({
    'Enrollment_ID': ["Unique identifier for each enrollment"],
    'Student_ID': ["Identifier for the enrolled student"],
    'Course_ID': ["Identifier for the enrolled course"],
    'DateKey': ["Key of the date of enrollment in YYYYMMDD format"],
    'Enrollment_Cost': ["Cost of enrollment for the course"]
})

# Updated DATE Table
date_table = pd.DataFrame({
    'Date_ID': ["Unique identifier for each date"],
    'Full_Date': ["Full date in YYYY-MM-DD format"],
    'Day': ["Day of the month"],
    'Month': ["Month name"],
    'Year': ["Year"],
    'Quarter': ["Quarter of the year"],
    'Weekday': ["Name of the day of the week"],
    'Is_Weekend': ["Boolean indicating if the day is a weekend"],
    'Fiscal_Quarter': ["Fiscal quarter"],
    'Fiscal_Year': ["Fiscal year"],
    'MonthKey': ["Month key in YYYYMM format"],
    'DateKey': ["Date key in YYYYMMDD format"]
})

# Creating a PDF document
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Function to add a table to the PDF
def add_table_to_pdf(df, title):
    pdf.set_font("Arial", style='B', size=14)
    pdf.cell(200, 10, title, ln=True, align='C')

    pdf.set_font("Arial", size=12)
    for column, description in df.items():
        pdf.cell(90, 10, column, border=1)
        pdf.cell(0, 10, description.values[0], border=1)
        pdf.ln(10)

# Adding each table to the PDF
add_table_to_pdf(country_df, "Country Table")
pdf.ln(10)
add_table_to_pdf(universities, "Universities Table")
pdf.ln(10)
add_table_to_pdf(professors, "Professors Table")
pdf.ln(10)
add_table_to_pdf(majors, "Majors Table")
pdf.ln(10)
add_table_to_pdf(courses, "Courses Table")
pdf.ln(10)
add_table_to_pdf(students, "Students Table")
pdf.ln(10)
add_table_to_pdf(enrollments, "Enrollments Table")
pdf.ln(10)
add_table_to_pdf(date_table, "Date Table")

# Outputting the PDF
pdf_output_path = "University_Data_Dictionary.pdf"
pdf.output(pdf_output_path)

pdf_output_path

'University_Data_Dictionary.pdf'

### <span style="color: orange;"> Hint: Snowflake Schema Diagram </span>

In [5]:
               #     Countries     Professors
               #        |          |
               #        +----+-----+
               #        |          |
               #     Universities  Courses
               #        |          |
               #        +----+-----+
               #             |
               #        Enrollments (Fact Table)
               #             |
               #        +----+-----+
               #        |          |
               #     Students     Date
               #        |
               #     Majors


---

## <span style="color: lightgreen;">**GOOD LUCK DATA WARRIORS!! 🤺**</span>